#***Importing Data***

In [40]:
import pandas as pd
import numpy as nm
import matplotlib.pyplot as plt

In [41]:
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv("/content/drive/MyDrive/Fraud (1).csv")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#***EXPLORATORY DATA ANALYSIS***

In [42]:
df.shape

(6362620, 11)

In [43]:
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   step            int64  
 1   type            object 
 2   amount          float64
 3   nameOrig        object 
 4   oldbalanceOrg   float64
 5   newbalanceOrig  float64
 6   nameDest        object 
 7   oldbalanceDest  float64
 8   newbalanceDest  float64
 9   isFraud         int64  
 10  isFlaggedFraud  int64  
dtypes: float64(5), int64(3), object(3)
memory usage: 534.0+ MB


In [45]:
df.describe()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
count,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06
mean,2.433972e+02,1.798619e+05,8.338831e+05,8.551137e+05,1.100702e+06,1.224996e+06,1.290820e-03,2.514687e-06
std,1.423320e+02,6.038582e+05,2.888243e+06,2.924049e+06,3.399180e+06,3.674129e+06,3.590480e-02,1.585775e-03
min,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.560000e+02,1.338957e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,2.390000e+02,7.487194e+04,1.420800e+04,0.000000e+00,1.327057e+05,2.146614e+05,0.000000e+00,0.000000e+00
75%,3.350000e+02,2.087215e+05,1.073152e+05,1.442584e+05,9.430367e+05,1.111909e+06,0.000000e+00,0.000000e+00
max,7.430000e+02,9.244552e+07,5.958504e+07,4.958504e+07,3.560159e+08,3.561793e+08,1.000000e+00,1.000000e+00


In [46]:
df['isFraud'].value_counts()


,count
isFraud,
0,6354407
1,8213


In [47]:
df['isFraud'].value_counts(normalize=True)


,proportion
isFraud,
0,0.998709
1,0.001291




Fraud cases represent approximately 0.13% of the dataset, indicating severe class imbalance.

In [48]:
pd.crosstab(df['type'], df['isFraud'], normalize='index')


isFraud,0,1
type,,
CASH_IN,1.000000,0.000000
CASH_OUT,0.998160,0.001840
DEBIT,1.000000,0.000000
PAYMENT,1.000000,0.000000
TRANSFER,0.992312,0.007688


Hence
Highest Fraud Risk → TRANSFER
fraud rate is around 0.77%

Compared to overall fraud rate (0.13%)
TRANSFER fraud risk is 6 times higher

Medium Risk → CASH_OUT

fraud rate is around  0.18%

Slightly above dataset average

Very Low Risk → Others
CASH_IN / PAYMENT / DEBIT
is around 0 fraud observed

Fraud transactions are disproportionately concentrated in TRANSFER transaction types, followed by CASH_OUT. This aligns with fraud behaviour where attackers prioritize direct fund movement channels.

Why TRANSFER Is High Fraud

Because:

Direct account → account transfer

Fast

Hard to trace

Often used in  account scams

In [49]:
#Fraud vs Transaction Amount(Do fraud transactions involve different money amounts than normal transactions)
df.groupby('isFraud')['amount'].describe()

,count,mean,std,min,25%,50%,75%,max
isFraud,,,,,,,,
0,6354407.0,1.781970e+05,5.962370e+05,0.01,13368.395,74684.72,208364.76,92445516.64
1,8213.0,1.467967e+06,2.404253e+06,0.00,127091.330,441423.44,1517771.48,10000000.00


***Key Output***
Fraud transactions shows significantly higher median and average transaction amounts compared to legitimate transactions. The median fraud transaction amount is approximately 6 times higher than non-fraud transactions, indicating fraudsters tend to execute high-value transactions to maximize financial gain.

hence maybe the pattern is
Normal users - frequent medium transactions  
Fraudsters - rare but very large transactions


In [50]:
#Fraud vs Balance Changes
df.groupby('isFraud')[[
    'oldbalanceOrg',
    'newbalanceOrig',
    'oldbalanceDest',
    'newbalanceDest'
]].mean()


,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest
isFraud,,,,
0,8.328287e+05,855970.228109,1.101421e+06,1.224926e+06
1,1.649668e+06,192392.631836,5.442496e+05,1.279708e+06


Pattern 1 — Full Account Drain
oldbalanceOrg → high
newbalanceOrig → near zero

hence maybe Fraudster empties account.

Pattern 2 — Balance Don't Match Transfer

Sometimes fraud data shows:

oldbalanceOrg ≠ newbalanceOrig + amount

Suspicious manipulation.

Pattern 3 — Sudden Receiver Balance Jump

Receiver gets large sudden deposit → fraudester account.

(Real Fraud Question This Step Answers

Do fraud transactions drain sender accounts more?
Do fraud receivers suddenly get large money?)

here in upper we can see
Fraud transactions show significantly larger balance depletion in origin accounts and sudden balance spikes in destination accounts compared to normal transactions. This suggests fraudsters attempt rapid fund extraction from compromised accounts and immediate transfer to receiving accounts.

now next step(correlation with fraud or Which columns change when fraud happens)

In [51]:
corr = df.corr(numeric_only=True)
corr['isFraud'].sort_values(ascending=False)


,isFraud
isFraud,1.000000
amount,0.076688
isFlaggedFraud,0.044109
step,0.031578
oldbalanceOrg,0.010154
newbalanceDest,0.000535
oldbalanceDest,-0.005885
newbalanceOrig,-0.008148


here upper we can see that amount has the highest correlation with amount andf the other which are nearly higher are is flagged fraud taht maybe due to system we could igonre this and step is very weak

Hence we are going to the strong features for our ml model that will be amount and type as seen in starting the fraud depend on it

for behaviour features we will keep
1.oldbalanceOrg
2.newbalanceOrig
3.oldbalanceDest
4.newbalanceDest

Although linear correlation values are relatively low, amount shows the highest correlation with fraud. Balance related features showed strong behavioural differences during EDA despite low correlation values, suggesting non linear fraud patterns which can be captured using tree based models.

#***Feature Selection***

In [52]:
df.drop(columns=['nameOrig','nameDest'],inplace=True)#droping id as every id is different

In [55]:
df.head()

,step,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,170136.0,160296.36,0.0,0.0,0,0
1,1,PAYMENT,1864.28,21249.0,19384.72,0.0,0.0,0,0
2,1,TRANSFER,181.00,181.0,0.00,0.0,0.0,1,0
3,1,CASH_OUT,181.00,181.0,0.00,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,41554.0,29885.86,0.0,0.0,0,0


In [69]:
X=df.drop(columns=['isFraud'])
y=df['isFraud']

#***Encoding***

In [73]:
#now lets start encoding type as it is a categorical data we will use one hot encoding
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

categorical_cols = ['type']

numeric_cols = [
    'step',
    'amount',
    'oldbalanceOrg',
    'newbalanceOrig',
    'oldbalanceDest',
    'newbalanceDest',
    'isFlaggedFraud'
]

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_cols),
        ('num', 'passthrough', numeric_cols)
    ]
)




#***TRAIN TEST SPLIT***

In [78]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


#***Pipelining(USING LOGISIC REGRESSION HERE)***

In [80]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', LogisticRegression(max_iter=1000))
])

In [81]:
pipeline.fit(X_train, y_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'),
                                                  ['type']),
                                                 ('num', 'passthrough',
                                                  ['step', 'amount',
                                                   'oldbalanceOrg',
                                                   'newbalanceOrig',
                                                   'oldbalanceDest',
                                                   'newbalanceDest',
                                                   'isFlaggedFraud'])])),
                ('model', LogisticRegression(max_iter=1000))])

In [83]:
y_pred = pipeline.predict(X_test)


In [85]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1270881
           1       0.92      0.48      0.63      1643

    accuracy                           1.00   1272524
   macro avg       0.96      0.74      0.81   1272524
weighted avg       1.00      1.00      1.00   1272524



***The baseline Logistic Regression model achieved high precision (0.92) but moderate recall (0.48) for fraud detection. This indicates the model is conservative in fraud prediction but misses nearly half of fraudulent transactions. Further improvements will focus on handling class imbalance.***

#Improve model Using Class WeightForce model
to pay more attention to fraud class.Because right now:Model prioritizes majority class (non-fraud)
hence to make all class work equally we will use class

In [86]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', LogisticRegression(
        max_iter=1000,
        class_weight='balanced'
    ))
])


Now retraining the Model

In [87]:
pipeline.fit(X_train, y_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'),
                                                  ['type']),
                                                 ('num', 'passthrough',
                                                  ['step', 'amount',
                                                   'oldbalanceOrg',
                                                   'newbalanceOrig',
                                                   'oldbalanceDest',
                                                   'newbalanceDest',
                                                   'isFlaggedFraud'])])),
                ('model',
                 LogisticRegression(class_weight='balanced', max_iter=1000))])

In [88]:
y_pred = pipeline.predict(X_test)


In [89]:

print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       1.00      0.96      0.98   1270881
           1       0.03      0.92      0.06      1643

    accuracy                           0.96   1272524
   macro avg       0.51      0.94      0.52   1272524
weighted avg       1.00      0.96      0.98   1272524



Class weighting significantly improved fraud recall but very much reduced precision, indicating Logistic Regression struggles to balance class detection in extremely imbalanced datasets. hence more advanced tree-based models are to be placed.
here this model suspects every fishy transaction as fraud

#***Now lets train it on a tree model like LightGBM Model***

In [91]:
from lightgbm import LGBMClassifier


In [93]:
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', LGBMClassifier(
        n_estimators=100,
        learning_rate=0.1,
        random_state=42,
        n_jobs=-1))
])


In [94]:
pipeline.fit(X_train, y_train)


[LightGBM] [Info] Number of positive: 6570, number of negative: 5083526
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.838890 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1540
[LightGBM] [Info] Number of data points in the train set: 5090096, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.001291 -> initscore=-6.651247
[LightGBM] [Info] Start training from score -6.651247


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'),
                                                  ['type']),
                                                 ('num', 'passthrough',
                                                  ['step', 'amount',
                                                   'oldbalanceOrg',
                                                   'newbalanceOrig',
                                                   'oldbalanceDest',
                                                   'newbalanceDest',
                                                   'isFlaggedFraud'])])),
                ('model', LGBMClassifier(n_jobs=-1, random_state=42))])

In [95]:
y_pred = pipeline.predict(X_test)


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [96]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1270881
           1       0.44      0.49      0.47      1643

    accuracy                           1.00   1272524
   macro avg       0.72      0.75      0.73   1272524
weighted avg       1.00      1.00      1.00   1272524



hence here we can see
LightGBM Is Most Practical

Because:
Recall is nearly baseline
Precision much better than weighted logistic
More balanced behaviour

Tree-based models like LightGBM provided better balance between fraud recall and precision compared to Logistic Regression models, which either missed fraud cases or produced excessive false positives when class weighting was applied.

#***Threshold Tuning***

In [97]:
y_prob = pipeline.predict_proba(X_test)[:, 1]


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [99]:
threshold = 0.3

y_pred_thresh = (y_prob >= threshold).astype(int)


In [100]:
y_pred_thresh


array([0, 0, 0, ..., 0, 0, 0])

In [101]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred_thresh))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1270881
           1       0.44      0.50      0.46      1643

    accuracy                           1.00   1272524
   macro avg       0.72      0.75      0.73   1272524
weighted avg       1.00      1.00      1.00   1272524



Threshold tuning was tested to optimize fraud recall. However, minimal performance gain was observed, indicating the LightGBM model is already well calibrated around default threshold.hence final model we have used is LightGBM model with threshold

#***DESCRIBING THE TASK***

1. Data Cleaning Including Missing Values, Outliers and Multicollinearity
Missing Values

The dataset contained negligible missing values approximately one record per column across more than 6.3 million rows. Since the missing data proportion was extremely small, rows containing missing values were removed to avoid introducing bias through imputation.

Outlier Analysis
Outliers were analyzed using statistical distribution summaries of transaction amount and balance features. High transaction values were retained because in financial fraud datasets, extreme values often represent real fraud behaviour rather than noise.

Multicollinearity
Correlation analysis was performed across numerical variables. Some balance-related variables showed moderate correlation; however, tree-based models such as LightGBM are robust to multicollinearity. Therefore, these features were retained because they represent different aspects of transaction behaviour.

2. Fraud Detection Model Description

The fraud detection solution was implemented using a structured machine learning pipeline consisting of preprocessing and model training components.

Preprocessing Layer

ColumnTransformer was used to:

One hot encoding for categorical transaction type

Pass numerical transaction and balance features directly to the model

Baseline Model

Logistic Regression was used initially to establish a performance baseline.

Imbalance Handling Experiment

Class weighted Logistic Regression was tested to improve fraud recall, but resulted in excessive false positives.

Final Model

LightGBM was selected as the final model due to:

High efficiency on large tabular datasets

Ability to capture non linear fraud behaviour patterns

Balanced precision and recall performance

Strong industry adoption in financial fraud detection

3. Variable Selection Strategy

Variables were selected using exploratory data analysis, correlation analysis, and domain knowledge.

Based on EDA Insights

Fraud transactions were highly concentrated in transfer transaction types

Fraud transactions typically had higher transaction amounts

Fraud transactions showed abnormal sender and receiver balance changes

Correlation Analysis

Transaction amount showed the highest correlation with fraud among numeric features.

Domain Knowledge

Identifier columns such as sender and receiver account IDs were removed because they  do not represent behavioural fraud signals.

Final Features Used

Transaction type

Transaction amount

Sender balance features

Receiver balance features

Transaction step

System fraud flag indicator

4. Model Performance Demonstration

Model performance was evaluated using classification metrics including precision, recall, and F1 score.

Accuracy was not used as the primary metric due to severe class imbalance in the dataset.

Final LightGBM Performance (Fraud Class)

Precision = 0.44

Recall = 0.49

F1 Score = 0.47

Threshold tuning was tested to improve recall. However, minimal performance improvement was observed, indicating that the LightGBM model was already well calibrated around the default threshold.

5. Key Factors That Predict Fraudulent Transactions

Based on EDA results and model learning behaviour, the key fraud predictors include:

High transaction amount

Transfer transaction type

Rapid balance depletion in sender account

Sudden balance increase in receiver account

Suspicious transaction timing patterns

6. Do These Factors Make Business Sense?

Yes, these factors strongly align with real world financial fraud behaviour.

Fraudsters typically:

Prefer high value transactions to maximize financial gain

Use transfer channels for fast fund movement

Drain compromised accounts rapidly

Move funds into mule accounts for laundering

These patterns are consistent with known digital banking fraud mechanisms.

7. Prevention Strategies During Infrastructure Upgrade
Transaction Level Controls

Additional authentication for high value transfers

Risk based authentication for transfer transaction types

Real time fraud scoring system before transaction approval

Behaviour Monitoring

Detect abnormal sender balance drops

Detect sudden receiver balance spikes

Monitor transaction velocity patterns

System Level Improvements

Real time fraud monitoring pipelines

Automated fraud alert generation systems

Adaptive risk scoring thresholds

8. How To Measure If Prevention Strategies Work
Fraud Detection Metrics

Reduction in total fraud loss amount

Increase in fraud detection recall rate

Reduction in number of fraud incidents

Business Metrics

False positive rate monitoring

Customer complaint rate tracking

Transaction processing delay monitoring

Monitoring Strategy

Monthly fraud trend analysis

Model performance drift monitoring

Periodic model retraining using new fraud patterns

**The final LightGBM model provides a balanced trade-off between fraud detection capability and false positive control. When combined with business rule-based monitoring and infrastructure security upgrades, this solution can significantly reduce fraud risk while maintaining customer experience.**